## **NLP - Text Classification Project**
Group H - August 2025

Classification of tweets from Twitter that have been manually tagged for sentiment analysis.

In [1]:
# # Needed for Google Colab
# !pip install --quiet evaluate transformers optuna datasets nltk scikit-learn
# !pip install numpy==1.26.4

In [2]:
%env CUDA_LAUNCH_BLOCKING=1

from wordcloud import WordCloud, STOPWORDS
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import Dataset
from optuna.pruners import MedianPruner
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import string
import time
import glob
import nltk
import evaluate
import transformers
import torch
import optuna
import wandb
wandb.login()

os.environ["WANDB_PROJECT"] = "tweet-sentiment-classification_split_to_test"
os.environ["WANDB_INIT_TIMEOUT"] = "180"

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

env: CUDA_LAUNCH_BLOCKING=1
Mounted at /content/drive


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hillas (hillas-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
# Load data
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_project/Data/Corona_NLP_train.csv', encoding='latin1')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_project/Data/Corona_NLP_test.csv', encoding='latin1')

In [4]:
# Merge and shuffle for better stratified splits
df_full = pd.concat([df_train, df_test], ignore_index=True)
df_full = df_full.sample(frac=1.0, random_state=42).reset_index(drop=True)

### Pre-processing the Data

The tweets were cleaned by lowercasing (to reduce redundancy), removing stopwords, punctuation, numbers, short words, and applying lemmatization to reduce words to their base form (e.g. running → run). <br>This helps reduce noise and improve model performance.

In [5]:
# Try without pre-processing
is_preprocessed = "no_preprocess"
df_full["clean_text"] = df_full["OriginalTweet"]

## Fine-Tuning Pretrained Language Models

Apply NLP techniques using transfer learning on our tweets dataset. Specifically, fine-tuning two pretrained transformer-based models from the Hugging Face library — BERT and RoBERTa — on our sentiment classification task. These models will be trained using both standard PyTorch and the Hugging Face API. Model performance will be monitored and tuned using hyperparameter optimization (Optuna) and experiment tracking (Weights & Biases).

**Load Pretrained Models**

Initialize tokenizers and models for BERT and RoBERTa, both widely used transformer architectures for text classification. The classification head is configured based on the number of sentiment labels.

In [6]:
# Choose pretrained models
bert_model_name = "bert-base-uncased"
roberta_model_name = "roberta-base"

sentiment_labels = df_full['Sentiment'].unique()
n_labels = len(sentiment_labels)

# Load BERT tokenizer and model
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=n_labels)

# Load RoBERTa tokenizer and model
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name, num_labels=n_labels)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Encode Sentiment Labels**

Map each unique sentiment label to a numeric ID for model compatibility, and apply this mapping to both training and validation datasets.

In [7]:
# Mapping sentiments to unique numeric IDs
unique_labels = sorted(df_full["Sentiment"].unique())
label2id = {label: idx for idx, label in enumerate(unique_labels)}
df_full["label"] = df_full["Sentiment"].map(label2id)

In [8]:
# Stratified split: 70% train, 15% val, 15% test
train_val_df, test_df = train_test_split(df_full, test_size=0.15, stratify=df_full["label"], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.1765, stratify=train_val_df["label"], random_state=42)

# Confirm sizes
print("Train size:", len(train_df))
print("Val size:", len(val_df))
print("Test size:", len(test_df))

Train size: 31466
Val size: 6745
Test size: 6744


**Use Small Subsets for Quick Evaluation**

Select shuffled samples from each training and validation dataset for both BERT and RoBERTa. This allows faster experimentation during model development.

In [9]:
train_subset_df, _ = train_test_split(
    train_df[["clean_text", "label"]],
    train_size=2000,
    stratify=train_df["label"],
    random_state=42
)

val_subset_df, _ = train_test_split(
    val_df[["clean_text", "label"]],
    train_size=500,
    stratify=val_df["label"],
    random_state=42
)

**Tokenization**

Define tokenization functions for BERT and RoBERTa to preprocess text with truncation and fixed padding.

Transform training and validation DataFrames into Dataset objects compatible with Hugging Face workflows.

Apply tokenization to training and validation datasets using each model's tokenizer.

In [10]:
# Tokenize function
def tokenize_function_bert(examples):
    return bert_tokenizer(examples["clean_text"], truncation=True, padding='max_length', max_length=64)

def tokenize_function_roberta(examples):
    return roberta_tokenizer(examples["clean_text"], truncation=True, padding='max_length', max_length=64)

In [11]:
# Convert DataFrame to Hugging Face Dataset
hf_subset_train = Dataset.from_pandas(train_subset_df)
hf_subset_val = Dataset.from_pandas(val_subset_df)

hf_train = Dataset.from_pandas(train_df[["clean_text", "label"]])
hf_val = Dataset.from_pandas(val_df[["clean_text", "label"]])
hf_test = Dataset.from_pandas(test_df[["clean_text", "label"]])

In [12]:
# Tokenize subsets
# Tokenize for BERT
tokenized_bert_train_sub = hf_subset_train.map(tokenize_function_bert, batched=True)
tokenized_bert_train_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_val_sub = hf_subset_val.map(tokenize_function_bert, batched=True)
tokenized_bert_val_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Tokenize for RoBERTa
tokenized_roberta_train_sub = hf_subset_train.map(tokenize_function_roberta, batched=True)
tokenized_roberta_train_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_roberta_val_sub = hf_subset_val.map(tokenize_function_roberta, batched=True)
tokenized_roberta_val_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [13]:
# Tokenize full dataset
# Tokenize for BERT
tokenized_bert_train = hf_train.map(tokenize_function_bert, batched=True)
tokenized_bert_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_val = hf_val.map(tokenize_function_bert, batched=True)
tokenized_bert_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_test = hf_test.map(tokenize_function_bert, batched=True)
tokenized_bert_test.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Tokenize for RoBERTa
tokenized_roberta_train = hf_train.map(tokenize_function_roberta, batched=True)
tokenized_roberta_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_roberta_val = hf_val.map(tokenize_function_roberta, batched=True)
tokenized_roberta_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_roberta_test = hf_test.map(tokenize_function_roberta, batched=True)
tokenized_roberta_test.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/31466 [00:00<?, ? examples/s]

Map:   0%|          | 0/6745 [00:00<?, ? examples/s]

Map:   0%|          | 0/6744 [00:00<?, ? examples/s]

Map:   0%|          | 0/31466 [00:00<?, ? examples/s]

Map:   0%|          | 0/6745 [00:00<?, ? examples/s]

Map:   0%|          | 0/6744 [00:00<?, ? examples/s]

In [14]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1_macro": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"]
    }

**Hyperparameters Tuning with Optuna**

In [15]:
def build_trainer(model_checkpoint, trial, run_prefix, train_dataset, val_dataset):
    # Sample hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_epochs = trial.suggest_int("num_train_epochs", 2, 5)
    n_samples = len(train_dataset)

    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=5)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    run_name = f"{run_prefix}-ep{num_epochs}-lr{learning_rate}-bs{batch_size}-samples{n_samples}-run{int(time.time())}-no_preprocess"

    args = TrainingArguments(
        output_dir=f"./results/{run_prefix}/{run_name}",
        disable_tqdm=True,
        fp16=True,
        eval_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        weight_decay=0.01,
        load_best_model_at_end=True,
        save_total_limit=1,
        logging_strategy="epoch",
        logging_dir=f"./logs/{run_prefix}/{run_name}",
        run_name=run_name,
        report_to="wandb",
        metric_for_best_model="f1_macro",
        greater_is_better=True
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    return trainer

In [16]:
def objective_bert(trial):
    trainer = build_trainer(
        model_checkpoint="bert-base-uncased",
        trial=trial,
        run_prefix="bert",
        train_dataset=tokenized_bert_train_sub,
        val_dataset=tokenized_bert_val_sub
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1_macro"]

def objective_roberta(trial):
    trainer = build_trainer(
        model_checkpoint="roberta-base",
        trial=trial,
        run_prefix="roberta",
        train_dataset=tokenized_roberta_train_sub,
        val_dataset=tokenized_roberta_val_sub
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1_macro"]

In [17]:
study_bert = optuna.create_study(direction="maximize",
                                 pruner=MedianPruner(n_startup_trials=2, n_warmup_steps=1),
                                 study_name=f"bert_study_stratify_{is_preprocessed}",
                                 storage=f"sqlite:////content/drive/MyDrive/Colab Notebooks/nlp_project/optuna/bert_study_stratify_{is_preprocessed}.db",
                                 load_if_exists=True)
study_bert.optimize(objective_bert, n_trials=5)

[I 2025-07-30 18:53:28,254] Using an existing study with name 'bert_study_stratify_no_preprocess' instead of creating a new one.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2841737310.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.5372, 'grad_norm': 6.685816287994385, 'learning_rate': 2.3489443779954745e-05, 'epoch': 1.0}
{'eval_loss': 1.4054003953933716, 'eval_accuracy': 0.384, 'eval_f1_macro': 0.32595609204930465, 'eval_runtime': 0.4379, 'eval_samples_per_second': 1141.772, 'eval_steps_per_second': 36.537, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.2228, 'grad_norm': 8.336163520812988, 'learning_rate': 1.76402937477921e-05, 'epoch': 2.0}
{'eval_loss': 1.1750056743621826, 'eval_accuracy': 0.472, 'eval_f1_macro': 0.45042814873368525, 'eval_runtime': 0.4151, 'eval_samples_per_second': 1204.435, 'eval_steps_per_second': 38.542, 'epoch': 2.0}


...

{'loss': 0.6484, 'grad_norm': 13.588672637939453, 'learning_rate': 1.2667568245226419e-05, 'epoch': 3.0}
{'eval_loss': 1.1044189929962158, 'eval_accuracy': 0.566, 'eval_f1_macro': 0.5792701719762192, 'eval_runtime': 0.7244, 'eval_samples_per_second': 690.182, 'eval_steps_per_second': 44.172, 'epoch': 3.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.3762, 'grad_norm': 7.414652347564697, 'learning_rate': 1.9948926370435305e-07, 'epoch': 4.0}
{'eval_loss': 1.173612356185913, 'eval_accuracy': 0.57, 'eval_f1_macro': 0.572138258244468, 'eval_runtime': 0.7379, 'eval_samples_per_second': 677.617, 'eval_steps_per_second': 43.367, 'epoch': 4.0}
{'train_runtime': 48.8494, 'train_samples_per_second': 163.769, 'train_steps_per_second': 10.236, 'train_loss': 0.8974813690185547, 'epoch': 4.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
[I 2025-07-30 19:00:05,066] Trial 14 finished with value: 0.5792701719762192 and parameters: {'learning_rate': 4.987231592608826e-05, 'batch_size': 16, 'num_train_epochs': 4}. Best is trial 2 with value: 0.6268387639451145.


{'eval_loss': 1.1044189929962158, 'eval_accuracy': 0.566, 'eval_f1_macro': 0.5792701719762192, 'eval_runtime': 0.7256, 'eval_samples_per_second': 689.105, 'eval_steps_per_second': 44.103, 'epoch': 4.0}


In [18]:
study_roberta = optuna.create_study(direction="maximize",
                                    pruner=MedianPruner(n_startup_trials=2, n_warmup_steps=1),
                                    study_name=f"roberta_study_stratify_{is_preprocessed}",
                                    storage=f"sqlite:////content/drive/MyDrive/Colab Notebooks/nlp_project/optuna/roberta_study_stratify_{is_preprocessed}.db",
                                    load_if_exists=True)
study_roberta.optimize(objective_roberta, n_trials=5)

[I 2025-07-30 19:00:05,883] Using an existing study with name 'roberta_study_stratify_no_preprocess' instead of creating a new one.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2841737310.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.4884, 'grad_norm': 20.117536544799805, 'learning_rate': 1.7151018577365895e-05, 'epoch': 1.0}
{'eval_loss': 1.2688637971878052, 'eval_accuracy': 0.438, 'eval_f1_macro': 0.3994078221606506, 'eval_runtime': 1.3542, 'eval_samples_per_second': 369.217, 'eval_steps_per_second': 46.521, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 1.1187, 'grad_norm': 20.31781768798828, 'learning_rate': 1.2880346620850482e-05, 'epoch': 2.0}
{'eval_loss': 1.1691570281982422, 'eval_accuracy': 0.516, 'eval_f1_macro': 0.520569629551196, 'eval_runtime': 1.376, 'eval_samples_per_second': 363.377, 'eval_steps_per_second': 45.786, 'epoch': 2.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

{'loss': 1.1731, 'grad_norm': 16.453243255615234, 'learning_rate': 5.504568653441014e-06, 'epoch': 2.0}
{'eval_loss': 1.1795964241027832, 'eval_accuracy': 0.486, 'eval_f1_macro': 0.501525241892156, 'eval_runtime': 1.3848, 'eval_samples_per_second': 361.059, 'eval_steps_per_second': 45.493, 'epoch': 2.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.9349, 'grad_norm': 25.773483276367188, 'learning_rate': 8.668612052663013e-08, 'epoch': 3.0}
{'eval_loss': 1.1954796314239502, 'eval_accuracy': 0.48, 'eval_f1_macro': 0.49387218204890015, 'eval_runtime': 1.3555, 'eval_samples_per_second': 368.86, 'eval_steps_per_second': 46.476, 'epoch': 3.0}
{'train_runtime': 67.9291, 'train_samples_per_second': 88.327, 'train_steps_per_second': 11.041, 'train_loss': 1.2083517252604166, 'epoch': 3.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
[I 2025-07-30 19:08:23,891] Trial 14 finished with value: 0.501525241892156 and parameters: {'learning_rate': 1.625364759874315e-05, 'batch_size': 8, 'num_train_epochs': 3}. Best is trial 3 with value: 0.5954185429373753.


{'eval_loss': 1.1795964241027832, 'eval_accuracy': 0.486, 'eval_f1_macro': 0.501525241892156, 'eval_runtime': 2.9572, 'eval_samples_per_second': 169.079, 'eval_steps_per_second': 21.304, 'epoch': 3.0}


In [19]:
best_trial_bert = study_bert.best_trial
print('Bert best trial on subset:')
print(best_trial_bert.params)
best_trial_roberta = study_roberta.best_trial
print('RoBerta best trial on subset:')
print(best_trial_roberta.params)

Bert best trial on subset:
{'learning_rate': 3.793274955606689e-05, 'batch_size': 32, 'num_train_epochs': 5}
RoBerta best trial on subset:
{'learning_rate': 2.2755931315738675e-05, 'batch_size': 8, 'num_train_epochs': 5}


In [20]:
final_trainer_bert = build_trainer(
    model_checkpoint="bert-base-uncased",
    trial=best_trial_bert,
    run_prefix=f"bert_final_stratify_{is_preprocessed}",
    train_dataset=tokenized_bert_train,
    val_dataset=tokenized_bert_val
)
final_trainer_bert.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2841737310.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.8073, 'grad_norm': 17.527660369873047, 'learning_rate': 3.037703927863893e-05, 'epoch': 1.0}
{'eval_loss': 0.5230451822280884, 'eval_accuracy': 0.803409933283914, 'eval_f1_macro': 0.8101932447816488, 'eval_runtime': 5.3138, 'eval_samples_per_second': 1269.348, 'eval_steps_per_second': 39.708, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.4017, 'grad_norm': 3.877439022064209, 'learning_rate': 2.2790489367425553e-05, 'epoch': 2.0}
{'eval_loss': 0.48625776171684265, 'eval_accuracy': 0.8300963676797628, 'eval_f1_macro': 0.8370858579249486, 'eval_runtime': 5.3789, 'eval_samples_per_second': 1253.98, 'eval_steps_per_second': 39.228, 'epoch': 2.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.1709, 'grad_norm': 18.35509490966797, 'learning_rate': 7.625099453445153e-06, 'epoch': 4.0}
{'eval_loss': 0.4989120364189148, 'eval_accuracy': 0.8625648628613788, 'eval_f1_macro': 0.8667745215062006, 'eval_runtime': 5.3, 'eval_samples_per_second': 1272.652, 'eval_steps_per_second': 39.812, 'epoch': 4.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.1087, 'grad_norm': 75.40140533447266, 'learning_rate': 3.85495422317753e-08, 'epoch': 5.0}
{'eval_loss': 0.5772148370742798, 'eval_accuracy': 0.8624166048925129, 'eval_f1_macro': 0.8670454116331431, 'eval_runtime': 5.4202, 'eval_samples_per_second': 1244.429, 'eval_steps_per_second': 38.929, 'epoch': 5.0}
{'train_runtime': 460.5337, 'train_samples_per_second': 341.625, 'train_steps_per_second': 10.683, 'train_loss': 0.34988824177563677, 'epoch': 5.0}


TrainOutput(global_step=4920, training_loss=0.34988824177563677, metrics={'train_runtime': 460.5337, 'train_samples_per_second': 341.625, 'train_steps_per_second': 10.683, 'train_loss': 0.34988824177563677, 'epoch': 5.0})

In [21]:
final_trainer_roberta = build_trainer(
    model_checkpoint="roberta-base",
    trial=best_trial_roberta,
    run_prefix=f"roberta_final_stratify_{is_preprocessed}",
    train_dataset=tokenized_roberta_train,
    val_dataset=tokenized_roberta_val
)
final_trainer_roberta.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2841737310.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.9185, 'grad_norm': 53.88134765625, 'learning_rate': 1.8211686363617602e-05, 'epoch': 1.0}
{'eval_loss': 0.7521365880966187, 'eval_accuracy': 0.7215715344699778, 'eval_f1_macro': 0.7308123155848059, 'eval_runtime': 18.1404, 'eval_samples_per_second': 371.821, 'eval_steps_per_second': 46.526, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.5828, 'grad_norm': 3.2724196910858154, 'learning_rate': 1.3662813870812087e-05, 'epoch': 2.0}
{'eval_loss': 0.6392106413841248, 'eval_accuracy': 0.7731653076352853, 'eval_f1_macro': 0.7834685926110598, 'eval_runtime': 18.1613, 'eval_samples_per_second': 371.394, 'eval_steps_per_second': 46.472, 'epoch': 2.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.3773, 'grad_norm': 0.354358971118927, 'learning_rate': 4.565068885201058e-06, 'epoch': 4.0}
{'eval_loss': 0.8021866679191589, 'eval_accuracy': 0.820459599703484, 'eval_f1_macro': 0.8272797948288126, 'eval_runtime': 18.1233, 'eval_samples_per_second': 372.172, 'eval_steps_per_second': 46.57, 'epoch': 4.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'loss': 0.3051, 'grad_norm': 0.08615262806415558, 'learning_rate': 1.5039507224433287e-08, 'epoch': 5.0}
{'eval_loss': 0.9032319784164429, 'eval_accuracy': 0.8240177909562639, 'eval_f1_macro': 0.8297178083523201, 'eval_runtime': 18.1697, 'eval_samples_per_second': 371.222, 'eval_steps_per_second': 46.451, 'epoch': 5.0}
{'train_runtime': 1597.0837, 'train_samples_per_second': 98.511, 'train_steps_per_second': 12.316, 'train_loss': 0.5278612611011375, 'epoch': 5.0}


TrainOutput(global_step=19670, training_loss=0.5278612611011375, metrics={'train_runtime': 1597.0837, 'train_samples_per_second': 98.511, 'train_steps_per_second': 12.316, 'train_loss': 0.5278612611011375, 'epoch': 5.0})

In [24]:
# Evaluate on unseen data
print(final_trainer_bert.evaluate(tokenized_bert_test))
print(final_trainer_roberta.evaluate(tokenized_roberta_test))

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.5950532555580139, 'eval_accuracy': 0.8555753262158956, 'eval_f1_macro': 0.8585850962234574, 'eval_runtime': 5.3972, 'eval_samples_per_second': 1249.533, 'eval_steps_per_second': 39.094, 'epoch': 5.0}
{'eval_loss': 0.5950532555580139, 'eval_accuracy': 0.8555753262158956, 'eval_f1_macro': 0.8585850962234574, 'eval_runtime': 5.3972, 'eval_samples_per_second': 1249.533, 'eval_steps_per_second': 39.094, 'epoch': 5.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.9640306830406189, 'eval_accuracy': 0.8121293001186239, 'eval_f1_macro': 0.8175345663826563, 'eval_runtime': 18.2918, 'eval_samples_per_second': 368.689, 'eval_steps_per_second': 46.086, 'epoch': 5.0}
{'eval_loss': 0.9640306830406189, 'eval_accuracy': 0.8121293001186239, 'eval_f1_macro': 0.8175345663826563, 'eval_runtime': 18.2918, 'eval_samples_per_second': 368.689, 'eval_steps_per_second': 46.086, 'epoch': 5.0}


In [26]:
final_trainer_bert.save_model(f"models/bert_final_stratify_{is_preprocessed}")
bert_tokenizer.save_pretrained(f"models/bert_final_stratify_{is_preprocessed}")
!cp -r models/bert_final_stratify_{is_preprocessed} "/content/drive/MyDrive/Colab Notebooks/nlp_project/models/w_test_split/bert_best_model_stratify_{is_preprocessed}"

final_trainer_roberta.save_model(f"models/roberta_final_stratify_{is_preprocessed}")
roberta_tokenizer.save_pretrained(f"models/roberta_final_stratify_{is_preprocessed}")
!cp -r models/roberta_final_stratify_{is_preprocessed} "/content/drive/MyDrive/Colab Notebooks/nlp_project/models/w_test_split/roberta_best_model_stratify_{is_preprocessed}"